In [ ]:
## Dependencies
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from textwrap import wrap
import json

In [ ]:
#############################
#### Tables of Interest ####
############################

# --select COUNT(*) as row_count 
# select *
# from access_actionsusers as aa;
# -- Columns: 
# -- id *
# -- action *
# -- exclude
# -- argument
# -- user_id *
# -- * columns of interest
# -- Count: 0

# --select COUNT(*) as row_count 
# select *
# from accounts_user_login_information as auli;
# -- Columns: 
# -- user_id *
# -- last_login_at
# -- current_long_at
# -- last_login_ip
# -- current_login_ip
# -- login_count *
# -- * columns of interest
# -- Count: 222

# --select COUNT(*) as row_count 
# select *
# from accounts_user as au;
# -- Columns: 
# -- created
# -- updated
# -- id *
# -- username *
# -- displayname
# -- email *
# -- password
# -- active
# -- confirmed_at
# -- version_id
# -- profile
# -- preferences
# -- * columns of interest
# -- Count: 908

# --select COUNT(*) as row_count 
# select *
# from communities_members as cm;
# -- Columns:
# -- created
# -- updated
# -- json
# -- version_id
# -- id *
# -- role *
# -- visible
# -- active
# -- community_id *
# -- user_id *
# -- group_id *
# -- request_id
# -- * columns of interest
# -- Count: 147

# --select COUNT(*) as row_count 
# select * 
# from communities_metadata as cmet;
# -- Columns:
# -- created
# -- updated
# -- id *
# -- json *
# -- version_id
# -- slug *
# -- bucket_id *
# -- * columns of interest
# -- Count: 42

# --select COUNT(*) as row_count
# select * 
# from rdm_parents_community as rpc;
# -- Columns: 
# -- community_id *
# -- record_id *
# -- request_id
# -- * columns of interest
# -- Count: 5,682

# --select COUNT(*) as row_count
# select *
# from rdm_parents_metadata as rpm;
# -- Columns: 
# -- created
# -- updated
# -- id *
# -- json *
# -- version_id *
# -- * columns of interest
# -- Count: 6,237

# --select COUNT(*) as row_count
# select *
# from rdm_versions_state as rvs;
# -- Columns: 
# -- latest_index *
# -- parent_id *
# -- latest_id *
# -- next_draft_id *
# -- * columns of interest
# -- Count: 6,237

# --select COUNT(*) as row_count
# select *
# from rdm_records_metadata as rrm;
# -- Columns: 
# -- created *
# -- updated *
# -- id *
# -- json *
# -- version_id *
# -- index *
# -- bucket_id *
# -- parent_id *
# -- * columns of interest
# -- Count: 6,230

# --select COUNT(*) as row_count
# select *
# from files_bucket as fb;
# -- Columns:
# -- created *
# -- updated *
# -- id *
# -- default_location
# -- default_storage_class
# -- size
# -- quota_size
# -- max_file_size
# -- locked
# -- deleted *
# -- * columns of interest
# -- Count: 6,307

# --select COUNT(*) as row_count
# select *
# from rdm_records_files as rrf;
# -- Columns:
# -- created *
# -- updated *
# -- id *
# -- json *
# -- version_id
# -- key *
# -- record_id *
# -- object_version_id *
# -- * columns of interest
# -- Count: 6,232


In [ ]:
#####################################
#### Relationships of interest ######
#####################################

######################
#### accounts_user ###
######################

## Required relationships(i.e. solid lines)
## accounts_user id primary --> accounts_user_login_infomration user_id foreign

## Optional relationships (i.e. dashed lines)
## accounts_user id primary --> communities_archivedinvitations user_id foreign
## accounts_user id primary --> community_members user_id foreign
## accounts_user id primary --> accounts_userrole user_id foreign
## accounts_user id primary --> oauth2server_client user_id foreign
## accounts_user id primary --> oauth2server_token user_id foreign
## accounts_user id primary --> oauthclient_remote account user_id foreign
## accounts_user id primary --> access_actionusers user_id foreign
## accounts_user id primary --> accounts_user_session_activity foreign
## accounts_user id primary --> accounts_useridentity id_user foreign


##############################
#### rdm_versions_state ####
##############################

## Required relationships (i.e. solid lines)
## rdm_versions_state parent_id primary --> rdm_parents_metadata id foreign

## Optional relationships (i.e. dashed lines)
## rdm_versions_state latest_id foreign --> rdm_drafts_metadata id primary
## rdm_versions_state next_draft_id foreign --> rdm_drafts_metadata id primary
## rdm_versions_state latest_id foreign --> rdm_records_metadata id primary
## rdm_versions_state next_draft_id foreign --> rdm_records_metadata id primary


##############################
#### rdm_parents_metadata ####
##############################

## Required relationships (i.e. solid lines)
## rdm_parents_metadata id primary <-- rdm_versions_state parent_id foreign
## rdm_parents_metadata id primary <-- rdm_parents_commnity record_id foreign

## Optional relationships (i.e. dashed lines)
## rdm_parents_metadata id primary <-- rdm_records_metadata parent_id foreign
## rdm_parents_metadata id primary <-- rdm_drafts_metadata parent_id foreign


##############################
#### rdm_records_metadata ####
##############################

## Optional relationships (i.e. dashed lines)
## rdm_records_metadata parent_id foreign --> rdm_parents_metadata id primary
## rdm_records_metadata bucket_id foreign --> rdm_files_bucket id primary
## rdm_records_metadata id primary <-- rdm_records_files record_id foreign
## rdm_records_metadata id primary <-- rdm_versions_state latest_id foreign
## rdm_records_metadata id primary <-- rdm_versions_state next_draft_id foreign

## https://inveniordm.docs.cern.ch/develop/architecture/records/



In [ ]:
#########################################
#### JOIN TABLES TO CREATE USER INFO ####
#########################################

### accounts_user_login_information user_id primary <-- accounts_user id foreign

## Creates table with each user, their information, and when they last logged in

with user_info as (
	select 
	auli.user_id 
	,auli.login_count 
	,au.id  user_id_2
	,au.username 
	,au.email 
	from accounts_user_login_information as auli
	left join accounts_user as au 
	on auli.user_id = au.id
) 
--select COUNT(*) as row_count 
select * 
from user_info; 
## Count: 222

####################################################
####JOIN TABLES TO CREATE COMMUNITY MEMBER INFO ####
###################################################

## communities_members community_id foreign --> communities_metadata id primary 



with community_member_info as (
	select 
	cm.id as community_metadata_id
	,cm.role  as user_role
	,cm.community_id 
	,cm.user_id as user_id_3
    ,cm.group_id 
    ,cmet.id as community_id_2
	,cmet.json 
	,cmet.slug
	,cmet.bucket_id
	from communities_members as cm
	left join communities_metadata as cmet
	on cm.community_id = cmet.id
) 
-- select COUNT(*) as row_count 
select * 
from community_member_info; 
## Count: 147

###########################################
####JOIN TABLES TO CREATE PARENTS INFO ####
###########################################
## rdm_parents_commuity record_id primary <-- rdm_parents_metadata id foreign


with parents_info as (
	select 
	rpc.community_id 
	,rpc.record_id 
	,rpm.id as record_id_2
	,rpm.json as rdm_parents_metadata_json
	,rpm.version_id
	from rdm_parents_community as rpc
	left join rdm_parents_metadata as rpm
	on rpc.record_id = rpm.id
) 
--select COUNT(*) as row_count 
select * 
from parents_info;
## COUNT 5,682

##################################################################################
#### JOIN TABLES TO CREATE USER INFO, COMMUNITY MEMBER INFO, AND PARENT INFO ####
#################################################################################

with user_info as (
	select 
	auli.user_id 
	,auli.login_count 
	,au.id  user_id_2
	,au.username 
	,au.email 
	from accounts_user_login_information as auli
	left join accounts_user as au 
	on auli.user_id = au.id
), 
community_member_info as (
	select 
	cm.id as community_metadata_id
	,cm.role  as user_role
	,cm.community_id 
	,cm.user_id as user_id_3
    ,cm.group_id 
    ,cmet.id as community_id_2
	,cmet.json 
	,cmet.slug
	,cmet.bucket_id
	from communities_members as cm
	left join communities_metadata as cmet
	on cm.community_id = cmet.id
),
parents_info as (
	select 
	rpc.community_id as community_id_3 
	,rpc.record_id 
	,rpm.id as record_id_2
	,rpm.json as rdm_parents_metadata_json
	,rpm.version_id as rdm_parents_metadata_version_id
	from rdm_parents_community as rpc
	left join rdm_parents_metadata as rpm
	on rpc.record_id = rpm.id
),
user_community_parent_info as (
	select *
	from user_info as ui
	left join community_member_info as cmi on ui.user_id = cmi.user_id_3
	left join parents_info as pi on cmi.community_id = pi.community_id_3
) 

## select COUNT(*) as row_count 
select * 
from user_community_parent_info;


####################################################
#### Join tables to create parent metadata info ####
###################################################
## rdm_version_states parent_id primary <-- rdm_parents_metadata id foreign

with parent_version_info as (
    select 
    rvs.latest_index as  version_state_latest_index
    ,rvs.parent_id 
    ,rvs.latest_id as  version_state_latest_id
    ,rvs.next_draft_id as  version_state_next_draft_id
    ,rpm.created as parent_metadata_created
    ,rpm.updated as parent_metadata_updated
    ,rpm.id as parent_id_2
    ,rpm.json as parent_metadata_json
    ,rpm.version_id as parent_metadata_version_id
    from rdm_versions_state as rvs 
    left join rdm_parents_metadata as rpm
    on rvs.parent_id = rpm.id 
) 
--select COUNT(*) as row_count 
select * 
from parent_version_info; 
## Count: 6,237



In [ ]:
#######################################################
#### rdm_records_metadata --> rmd_parents_metadata ####
#######################################################

## rdm_version_states parent_id primary <-- rdm_parents_metadata id foreign ## called parent_metadata_info

## rdm_records_metadata parent_id primary <-- rdm_parents_metadata parent_id foreign

## rdm_records_files id primary <-- rdm_records_metadata id foreign ## called record_metadata_info



################################################
#### accounts_user --> rdm_records_metadata ####
################################################

## accounts_user_login_information user_id primary <-- accounts_user id foreign ## called user_info
## communities_members user_id foreign --> accounts_user id primary
## communities_members community_id foreign --> communities_metadata id primary ## called community info
## communities_metadata id foreign --> rdm_parents_community community_id primary 
## rdm_parents_commuity record_id primary <-- rdm_parents_metadata id foreign ## called parents_info


In [ ]:
## Load data from Prism database

## File to Load
file_path_1 = "data_prism/communities_files_202306271249.csv"
file_path_2 = "data_prism/rdm_drafts_files_202306271246.csv"
file_path_3 = "data_prism/rdm_drafts_metadata_202306271245.csv"
file_path_4 = "data_prism/rdm_parents_metadata_202306271245.csv"
file_path_5 = "data_prism/rdm_records_files_202306271245.csv"
file_path_6 = "data_prism/rdm_records_metadata_202306271246.csv"
file_path_7 = "data_prism/rdm_records_metadata_version_202306271246.csv"
file_path_8 = "data_prism/rdm_versions_state_202306271247.csv"
file_path_9 = "data_prism/subject_metadata_202306271248.csv"

 
## Read the CSV file and store into Pandas DataFrame
## Other encoding options: encoding="utf-8
load_communities_files_df = pd.read_csv(file_path_1, encoding="ISO-8859-1")
load_drafts_files_df = pd.read_csv(file_path_2, encoding="ISO-8859-1")
load_drafts_metadata_df = pd.read_csv(file_path_3, encoding="ISO-8859-1")
load_parents_metadata_df = pd.read_csv(file_path_4, encoding="ISO-8859-1")
load_records_filess_df = pd.read_csv(file_path_5, encoding="ISO-8859-1")
load_records_metadata_df = pd.read_csv(file_path_6, encoding="ISO-8859-1")
load_records_metadata_version_df = pd.read_csv(file_path_7, encoding="ISO-8859-1")
load_versions_state_df = pd.read_csv(file_path_8, encoding="ISO-8859-1")
load_subject_metadata_df = pd.read_csv(file_path_9, encoding="ISO-8859-1")


## Print dataframe
# load_communities_files_df.head()
# load_drafts_files_df.head()
# load_drafts_metadata_df.head()
# load_parents_metadata_df.head()
# load_records_filess_df.head()
load_records_metadata_df.head()
# load_records_metadata_version_df.head()
# load_versions_state_df.head()
# load_subject_metadata_df.head()




In [ ]:
# Flatten the JSON column
df_flattened = pd.json_normalize(load_records_metadata_df['json'].apply(json.loads))

# Merge the flattened data with the original DataFrame
df = pd.concat([load_records_metadata_df, df_flattened], axis=1)

# Drop the original JSON column if needed
df = df.drop('json', axis=1)

# Print the resulting DataFrame
df.head(50)


In [ ]:
# List the columns
column_names = df.columns.tolist()
print(column_names)